## Airbnb listings in New York City
<p><img src="https://assets.datacamp.com/production/project_1230/img/nyc.jpg" alt="New York City skyline" width="600px"></p>
<p>Welcome to New York City, one of the most-visited cities in the world. As a result, there are many <a href="https://www.airbnb.com/"><em>Airbnb</em></a> listings in New York City to meet the high demand for temporary lodging for anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like <code>.csv</code>, <code>.tsv</code>, and <code>.xlsx</code> (Excel files).</p>
<p>A <strong>CSV</strong>, or comma-separated-values, file is one of the most common ways that tabular data is stored. In a CSV file, each value is separated by a comma and each row is separated by a newline. Since this file format is so widely used and is non-proprietary, it's great for sharing data with others and can be parsed by a variety of software.</p>
<pre><code>"listing_id","price","nbhood_full"
2595,"225 dollars","Manhattan, Midtown"
3831,"89 dollars","Brooklyn, Clinton Hill"
5099,"200 dollars","Manhattan, Murray Hill"
</code></pre>
<p>A <strong>TSV</strong>, or tab-separated-values, file is similar to a CSV file, but tabs are used to separate values rather than commas:</p>
<pre><code>listing_id    host_name   last_review
2595    Jennifer    May 21 2019
3831    LisaRoxanne July 05 2019
5099    Chris   June 22 2019
</code></pre>
<p><strong>Excel files</strong> are often used by spreadsheet users. Excel files contain information about formatting and formulas created in Excel, but these things aren't usually necessary when working with data in R. Excel files can also contain multiple tables, so these files need to be imported carefully to make sure the correct table is used. The Excel file format is also proprietary, so there's a more limited pool of software that can read it.</p>
<p><img src="https://assets.datacamp.com/production/project_1230/img/airbnb_room_type.png" alt="Excel file" width="700px"></p>
<p>The three files that are available contain data on 2019 Airbnb listings. Here are the details:</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
<div style="font-size:20px"><b>datasets/airbnb_price.csv</b></div>
This is a CSV file containing data on the prices and neighborhoods of Airbnbs.
<ul>
<li><b><code>listing_id</code>:</b> unique identifier of listing</li>
<li><b><code>price</code>:</b> nightly listing price in USD</li>
<li><b><code>nbhood_full</code>:</b> name of borough and neighborhood where listing is located</li>
</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
<div style="font-size:20px"><b>datasets/airbnb_room_type.xlsx</b></div>
This is an Excel file containing data on Airbnb listing descriptions and room types.
<ul>
<li><b><code>listing_id</code>:</b> unique identifier of listing</li>
<li><b><code>description</code>:</b> listing description</li>
<li><b><code>room_type</code>:</b> Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments</li>
</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
<div style="font-size:20px"><b>datasets/airbnb_last_review.tsv</b></div>
This is a TSV file containing data on Airbnb host names and review dates.
<ul>
<li><b><code>listing_id</code>:</b> unique identifier of listing</li>
<li><b><code>host_name</code>:</b> name of listing host</li>
<li><b><code>last_review</code>:</b> date when the listing was last reviewed</li>
</ul>
</div>
<p><strong>Note:</strong> This project lets you apply the skills from the <a href="https://learn.datacamp.com/skill-tracks/importing-cleaning-data-with-r">Importing and Cleaning Data with R skill track</a>, including importing from different file types and cleaning numerical, categorical, and date data. We recommend that you take the courses in this track before starting this project.</p>

In [2]:
# # Load the necessary packages
library(readr)
library(readxl)
library(dplyr)
library(stringr)

# Import CSV for prices
airbnb_price <- read_csv('datasets/airbnb_price.csv')

# Import TSV for room types
airbnb_room_type <- read_excel('datasets/airbnb_room_type.xlsx')

# Import Excel file for review dates
airbnb_last_review <- read_tsv('datasets/airbnb_last_review.tsv')

# Join the three data frames together into one
listings <- airbnb_price %>%
  inner_join(airbnb_room_type, by = "listing_id") %>%
  inner_join(airbnb_last_review, by = "listing_id")

# Question 1: What is the average listing price? 
# Check column data types
glimpse(listings)

# To convert price to numeric, remove "dollars" from each value
avg_price <- listings %>%
  mutate(price_clean = str_remove(price, " dollars") %>%
        as.numeric()) %>%
  # Take the mean of price_clean
  summarize(avg_price = mean(price_clean)) %>%
  # Convert from a tibble to a single number
  as.numeric()

avg_price


# Question 2: How many of the listings are private rooms? 
# Check categories in room_type by counting the number of each
listings %>%
  count(room_type)

# Since there are differences in capitalization, make capitalization consistent
private_room_count <- listings %>%
  mutate(room_type = str_to_lower(room_type)) %>%
  # Then count the number of each room_type
  count(room_type) %>%
  # Get row containing count for private rooms only
  filter(room_type == "private room") 

# Extract number of rooms
nb_private_rooms <- private_room_count$n
nb_private_rooms


# Question 3: Which listing was most recently reviewed? 

# In order to use a function like max()/min() on the last_review column, it needs to be converted to Date
review_dates <- listings %>%
  # Convert to date using the format 'Month DD YYYY'
  mutate(last_review_date = as.Date(last_review, format = "%B %d %Y")) %>%
  # Use max() and min() to take the latest and earliest dates
  summarize(first_reviewed = min(last_review_date),
            last_reviewed = max(last_review_date))

review_dates


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



── Column specification ────────────────────────────────────────────────────────
cols(
  listing_id = col_double(),
  price = col_character(),
  nbhood_full = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  listing_id = col_double(),
  host_name = col_character(),
  last_review = col_character()
)




Rows: 25,209
Columns: 7
$ listing_id  <dbl> 2595, 3831, 5099, 5178, 5238, 5295, 5441, 5803, 6021, 6848…
$ price       <chr> "225 dollars", "89 dollars", "200 dollars", "79 dollars", …
$ nbhood_full <chr> "Manhattan, Midtown", "Brooklyn, Clinton Hill", "Manhattan…
$ description <chr> "Skylit Midtown Castle", "Cozy Entire Floor of Brownstone"…
$ room_type   <chr> "Entire home/apt", "Entire home/apt", "Entire home/apt", "…
$ host_name   <chr> "Jennifer", "LisaRoxanne", "Chris", "Shunichi", "Ben", "Le…
$ last_review <chr> "May 21 2019", "July 05 2019", "June 22 2019", "June 24 20…


[1] 141.7779

room_type,n
<chr>,<int>
entire home/apt,2665
Entire home/apt,8458
ENTIRE HOME/APT,2143
private room,2248
Private room,7241
PRIVATE ROOM,1867
shared room,110
Shared room,380
SHARED ROOM,97


[1] 11356

first_reviewed,last_reviewed
<date>,<date>
2019-01-01,2019-07-09
